# Examles of error propigation
Examples are taken from http://ipl.physics.harvard.edu/wp-uploads/2013/03/PS3_Error_Propagation_sp13.pdf and used on MCMC to show how the answers work

## Example 1

Example: suppose you measure the height H of a door and get 2.00 ± 0.03 m. This means that
H = 2.00 m and δH = 0.03 m. The door has a knob which is a height h = 0.88 ± 0.04 m from the bottom
of the door. Then the distance from the doorknob to the top of the door is Q = H − h = 1.12 m. What
is the uncertainty in Q?

Q = 1.12 ± 0.05 m


In [1]:
import numpy as np
import pymc3 as mc

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/Users/blarsen/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
model = mc.Model()

with model:

    H = mc.Normal('H', 2.00, tau=(0.03)**-2)
    h = mc.Normal('h', 0.88, tau=(0.04)**-2)
    Q = mc.Deterministic('Q', H-h)
    start = mc.find_MAP()
    trace = mc.sample(1e4, njobs=4, start=start)

    

INFO (theano.gof.compilelock): Waiting for existing lock by process '41098' (I am process '41205')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/blarsen/.theano/compiledir_Darwin-15.6.0-x86_64-i386-64bit-i386-3.6.5-64/lock_dir



You can find the C code in this temporary file: /var/folders/5r/3c2nfrr12vz53wrg2tmps2r80000gp/T/theano_compilation_error_aou5_pjj


Exception: ("Compilation failed (return status=1): ld: unexpected token: !tapi-tbd-v2 file '/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX10.12.sdk/usr/lib/libSystem.tbd' for architecture x86_64. clang-4.0: error: linker command failed with exit code 1 (use -v to see invocation). ", '[Elemwise{true_div,no_inplace}(TensorConstant{1.0}, TensorConstant{1111.111111111111})]')

In [ ]:
mc.traceplot(trace, combined=True, lines={'H':2, 'h':0.88, 'Q':2-0.88})
mc.summary(trace)
print("MCMC gives {0:.2f} +/- {1:.2f}, analytic gives {2} +/- {3}".format(trace['Q'].mean(), 
                                                                          trace['Q'].std(), 1.12, 0.05))

## Example 2

Example: a bird flies a distance d = 120 ± 3 m during a time t = 20.0 ± 1.2 s. The average speed of
the bird is v = d/t = 6 m/s. What is the uncertainty of v?

0.39 m/s.

In [ ]:
with mc.Model() as model:
    d = mc.Normal('d', 123, tau=(3)**-2)
    t = mc.Normal('t', 20.0, tau=(1.2)**-2)
    v = mc.Deterministic('v', d/t)
    start = mc.find_MAP()
    trace = mc.sample(40000, njobs=4, start=start)

In [ ]:
mc.summary(trace)
mc.traceplot(trace, combined=True, lines={'d':123, 't':20, 'v':6})
print("MCMC gives {0:.2f}, analytic gives {1}".format(trace['v'].std(), 0.39))

## Example 3

Example: the period of an oscillation is measured to be T = 0.20 ± 0.01 s. Thus the frequency is
f = 1/T = 5 Hz. What is the uncertainty in f? Answer: the percent uncertainty in T was 0.01/0.20 = 5%.
Thus the percent uncertainty in f is also 5%, which means that δf = 0.25 Hz. So f = 5.0 ± 0.3 Hz (after
rounding).

f = 5.0 ± 0.3 Hz

In [ ]:
with mc.Model() as model:
    T = mc.Normal('T', 0.20, tau=(0.01)**-2)

    mc.Deterministic('1/T', 1/T)
    start = mc.find_MAP()

    trace = mc.sample(1e4, burn=100, burn_till_tuned=True, start=start)


In [ ]:
mc.traceplot(trace, combined=True)

In [ ]:
print(mc.summary(trace))
print("MCMC gives {0:.1f} +/- {1:.1f}, analytic gives {2} +/- {3}".format(np.mean(trace['1/T']), 
                                                                          np.std(trace['1/T']), 
                                                                          5.0, 0.3))


## Example 4

Example: a ball is tossed straight up into the air with initial speed v0 = 4.0 ± 0.2 m/s. After a time
t = 0.60±0.06 s, the height of the ball is y = v0t−
1
2
gt2 = 0.636 m. What is the uncertainty of y? Assume
g = 9.80 m/s2
(no uncertainty in g).

Thus y would be properly reported as 0.6 ± 0.4 m.

In [ ]:
with mc.Model() as model:
    g = 9.80
    t = mc.Normal('t', 0.60, tau=(0.06)**-2)
    v0 = mc.Normal('v0', 4.0, tau=(0.2)**-2)

    h = mc.Deterministic('h', v0*t - 0.5*g*t**2)
    start = mc.find_MAP()

    trace = mc.sample(1e4, burn=100, burn_till_tuned=True, start=start)



In [ ]:
mc.traceplot(trace, combined=True)

In [ ]:
print(mc.summary(trace))
print("MCMC gives {0:.1f} +/- {1:.1f}, analytic gives {2} +/- {3}".format(np.mean(trace['h']), 
                                                                          np.std(trace['h']), 
                                                                          0.6, 0.4))


## Example 5
For example, suppose Ann and Billy both measure the speed of a moving ball. Ann measures 3.6±0.2 m/s
and Billy gets 3.3 ± 0.3 m/s. Do the two measurements agree?

D = 0.3 ± 0.4 m/s  so 0 is in the range, they do agree.

In [ ]:
with mc.Model() as model:

    A = mc.Normal('A', 3.6, tau=(0.2)**-2)
    B = mc.Normal('B', 3.3, tau=(0.3)**-2)

    D = mc.Deterministic('D', A-B)
    start = mc.find_MAP()

    trace = mc.sample(1e4, burn=100, burn_till_tuned=True, start=start)



In [ ]:
mc.traceplot(trace, combined=True);

In [ ]:
print(mc.summary(trace))
print("MCMC gives {0:.1f} +/- {1:.1f}, analytic gives {2} +/- {3}".format(np.mean(trace['D']), 
                                                                          np.std(trace['D']), 
                                                                          0.3, 0.4))
# mc.Matplot.plot(model)